## 6. Out-of-Core Learning

### Definition
**Out-of-Core Learning** (also called **External Memory Learning**) is the technique of training machine learning models on datasets that **cannot fit entirely into RAM**. Data is processed in chunks from disk storage.

### The Problem It Solves

Modern datasets are HUGE:
- Netflix: billions of movies ratings
- Google: terabytes of search queries daily
- Financial institutions: petabytes of transaction data
- Scientific simulations: multi-gigabyte datasets

These cannot fit in standard computer memory!

```
Without Out-of-Core Learning:
❌ Cannot load 1TB dataset into 16GB RAM
❌ Must use expensive hardware or give up

With Out-of-Core Learning:
✅ Process data in manageable chunks
✅ Use standard hardware
✅ Train on massive datasets economically
```

### How Out-of-Core Learning Works

#### Approach 1: Streaming Data Access
```
Load chunk 1 → Process → Update model → Discard chunk 1
Load chunk 2 → Process → Update model → Discard chunk 2
Load chunk 3 → Process → Update model → Discard chunk 3
...
Final model trained on ALL data without loading all at once
```

#### Approach 2: Incremental Learning
```
Model state (parameters) stays in memory
Only current batch loaded from disk
Update model with batch
Discard batch, load next batch
```

### Python Examples: Out-of-Core Learning


In [ ]:
import pandas as pd
import numpy as np
from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import StandardScaler
import sys

# Simulate huge CSV file (normally too large to load)
# In real world, this might be 100GB+ file

class HugeDataStreamSimulator:
    """Simulates reading huge CSV file in chunks"""
    
    def __init__(self, filename, chunksize=1000):
        self.filename = filename
        self.chunksize = chunksize
    
    def read_chunks(self):
        """Generator that yields chunks of data without loading entire file"""
        # In real scenario: pd.read_csv(filename, chunksize=self.chunksize)
        total_rows = 100_000
        for i in range(0, total_rows, self.chunksize):
            # Simulate reading chunk from disk
            X_chunk = np.random.rand(min(self.chunksize, total_rows - i), 50)
            y_chunk = np.random.randint(0, 2, min(self.chunksize, total_rows - i))
            yield X_chunk, y_chunk

# Method 1: Out-of-Core Learning with SGDClassifier
print("Method 1: Out-of-Core Learning with Incremental Training")
print("=" * 60)

model = SGDClassifier(loss='log', random_state=42, warm_start=False)
scaler = StandardScaler()

data_stream = HugeDataStreamSimulator(filename='huge_data.csv', chunksize=5000)

samples_processed = 0
batches_processed = 0

for X_chunk, y_chunk in data_stream.read_chunks():
    # Update statistics for scaling
    if batches_processed == 0:
        scaler.fit(X_chunk)
    
    X_scaled = scaler.transform(X_chunk)
    
    # Partial fit: update model with current chunk only
    if batches_processed == 0:
        model.partial_fit(X_scaled, y_chunk, classes=[0, 1])
    else:
        model.partial_fit(X_scaled, y_chunk)
    
    samples_processed += len(X_chunk)
    batches_processed += 1
    
    if batches_processed % 5 == 0:
        accuracy = model.score(X_scaled, y_chunk)
        print(f"Processed {samples_processed:,} samples ({batches_processed} batches), "
              f"Current accuracy: {accuracy:.4f}")

print(f"\nTotal samples processed: {samples_processed:,}")
print(f"Total batches: {batches_processed}")
print(f"Memory usage: Only one batch (~{5000 * 50 * 8 / (1024**2):.1f} MB) at a time")

# Method 2: Using Pandas read_csv with chunking (Real-world approach)
print("\n\nMethod 2: Real-world CSV Chunking")
print("=" * 60)

# Create sample CSV file
sample_df = pd.DataFrame({
    'feature_1': np.random.rand(10000),
    'feature_2': np.random.rand(10000),
    'feature_3': np.random.rand(10000),
    'target': np.random.randint(0, 2, 10000)
})
sample_df.to_csv('large_sample.csv', index=False)

# Read and train on chunks
model2 = SGDClassifier(loss='log', random_state=42, warm_start=False)

chunk_size = 1000
chunk_count = 0

for chunk in pd.read_csv('large_sample.csv', chunksize=chunk_size):
    X = chunk.drop('target', axis=1).values
    y = chunk['target'].values
    
    if chunk_count == 0:
        model2.partial_fit(X, y, classes=[0, 1])
    else:
        model2.partial_fit(X, y)
    
    chunk_count += 1

print(f"Trained on {chunk_count} chunks from CSV file")
print(f"Final accuracy: {model2.score(X, y):.4f}")


### Tools for Out-of-Core Learning


In [ ]:
# 1. Apache Spark: Distributed out-of-core learning
# pip install pyspark

from pyspark.sql import SparkSession
from pyspark.ml.classification import LogisticRegression

spark = SparkSession.builder.appName("OutOfCore").getOrCreate()

# Read huge CSV into Spark DataFrame (handles out-of-core automatically)
df = spark.read.csv("huge_data.csv", header=True, inferSchema=True)

# Train on data that might be larger than single machine RAM
lr = LogisticRegression(maxIter=100, regParam=0.01)
model = lr.fit(df)

predictions = model.transform(df)
predictions.select("prediction", "label").show()

# 2. Dask: Parallel pandas for out-of-core data
# pip install dask[dataframe]

import dask.dataframe as dd
from dask.array import from_delayed
import dask.array as da

# Load huge CSV with Dask (lazy evaluation, doesn't load all at once)
ddf = dd.read_csv("huge_data.csv")

# Compute statistics without loading all data
mean_values = ddf.mean().compute()
print(f"Mean of columns: {mean_values}")

# 3. Scikit-learn with partial_fit
# Already demonstrated above

# 4. TensorFlow with tf.data API
# pip install tensorflow

import tensorflow as tf

def create_dataset(features, labels, batch_size=1000):
    """Create dataset that yields batches for out-of-core training"""
    dataset = tf.data.Dataset.from_tensor_slices((features, labels))
    dataset = dataset.shuffle(buffer_size=10000)
    dataset = dataset.batch(batch_size)
    return dataset

# Train model on huge dataset
# model.fit(dataset)  # Automatically handles batching


### Advantages of Out-of-Core Learning

#### 1. **Handles Unlimited Data Size** 📊


In [ ]:
# Can train on terabytes even with gigabytes of RAM
# Example: 1TB dataset with 16GB RAM

# Process 1TB dataset:
# - Read 100MB chunks
# - Process 100MB chunk
# - Update model
# - Total memory needed: ~200MB (model + batch)
# - Train on 1TB with 16GB RAM easily!


#### 2. **Cost-Effective** 💰
- No need for expensive supercomputers
- Regular hardware sufficient
- Can process on edge devices
- Reduces infrastructure costs

#### 3. **Scalable** 📈
- Easily scales to growing datasets
- No need to rebuild system for larger data
- Seamlessly handle unlimited data streams

#### 4. **Efficient** ⚡
- Memory usage remains constant regardless of dataset size
- Computation spreads over time
- Can prioritize resources

### Disadvantages of Out-of-Core Learning

#### 1. **Slower Training** 🐢
- I/O operations (disk read) are slow
- Multiple passes through data needed
- Slower than in-memory training
- Disk bandwidth is limiting factor


In [ ]:
# Comparison: In-Core vs Out-of-Core
import time

# In-Core: Load all data at once
X_full = np.random.rand(100_000, 100)
y_full = np.random.randint(0, 2, 100_000)

start = time.time()
model_in_core = SGDClassifier(max_iter=10)
model_in_core.fit(X_full, y_full)
in_core_time = time.time() - start
print(f"In-Core training: {in_core_time:.2f} seconds")

# Out-of-Core: Process in chunks from disk
model_out_core = SGDClassifier(max_iter=1, warm_start=False)
start = time.time()

for i in range(0, len(X_full), 5000):
    X_chunk = X_full[i:i+5000]
    y_chunk = y_full[i:i+5000]
    
    if i == 0:
        model_out_core.partial_fit(X_chunk, y_chunk, classes=[0, 1])
    else:
        model_out_core.partial_fit(X_chunk, y_chunk)

out_core_time = time.time() - start
print(f"Out-of-Core training: {out_core_time:.2f} seconds")
print(f"Overhead: {out_core_time/in_core_time:.2f}x slower")


#### 2. **Implementation Complexity** 🔧
- More code to write
- Error handling more involved
- Debugging harder (data constantly changing)
- State management required

#### 3. **Limited Model Options**
- Not all algorithms support incremental learning
- Deep learning models harder to implement out-of-core
- Some models require seeing all data

### When to Use Out-of-Core Learning

✅ **Use Out-of-Core when:**
- Dataset too large for available RAM
- Streaming data constantly arriving
- Cannot afford powerful hardware
- Training on edge devices
- Processing real-time data streams

❌ **Don't use when:**
- Data fits comfortably in RAM
- Speed is critical
- Need maximum model accuracy
- Using complex deep learning models

### Real-World Example: Processing Million-Row Dataset


In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import SGDClassifier
import gc

class OutOfCoreMLPipeline:
    """Complete out-of-core ML pipeline"""
    
    def __init__(self, model_type='classification'):
        self.model = SGDClassifier(loss='log', n_jobs=-1) if \
                    model_type == 'classification' else SGDRegressor()
        self.scaler = StandardScaler()
        self.label_encoder = LabelEncoder()
        self.is_fitted = False
        self.feature_names = None
        
    def fit_on_csv(self, csv_filepath, chunksize=10000, 
                   target_column='target', categorical_columns=None):
        """Train model on huge CSV file chunk by chunk"""
        
        categorical_columns = categorical_columns or []
        chunks_processed = 0
        
        for chunk in pd.read_csv(csv_filepath, chunksize=chunksize):
            # Separate features and target
            y = chunk[target_column].values
            X = chunk.drop(target_column, axis=1)
            
            # Handle categorical features
            for col in categorical_columns:
                if col in X.columns:
                    X[col] = self.label_encoder.fit_transform(X[col])
            
            # Convert to numpy
            X = X.values
            
            # Scale features
            if not self.is_fitted:
                self.scaler.fit(X)
                self.feature_names = X.shape[1]
            
            X_scaled = self.scaler.transform(X)
            
            # Update model
            if not self.is_fitted:
                self.model.partial_fit(X_scaled, y, classes=np.unique(y))
                self.is_fitted = True
            else:
                self.model.partial_fit(X_scaled, y)
            
            chunks_processed += 1
            if chunks_processed % 10 == 0:
                acc = self.model.score(X_scaled, y)
                print(f"Processed {chunks_processed*chunksize:,} rows, "
                      f"Current accuracy: {acc:.4f}")
            
            # Free memory
            gc.collect()
        
        print(f"\nTotal chunks processed: {chunks_processed}")
        return self
    
    def predict_on_csv(self, csv_filepath, output_filepath, chunksize=10000):
        """Make predictions on huge CSV file"""
        
        predictions_list = []
        
        for chunk in pd.read_csv(csv_filepath, chunksize=chunksize):
            X = chunk.values
            X_scaled = self.scaler.transform(X)
            preds = self.model.predict(X_scaled)
            predictions_list.append(preds)
            
            gc.collect()
        
        # Save predictions
        all_predictions = np.concatenate(predictions_list)
        np.save(output_filepath, all_predictions)
        print(f"Predictions saved to {output_filepath}")
        return all_predictions

# Usage
pipeline = OutOfCoreMLPipeline()
# pipeline.fit_on_csv('million_rows.csv', chunksize=50000, 
#                     target_column='label')


---
